# Database Check
Quick validation of database health after repopulation.

| Cell | Description |
|------|-------------|
| 1 | Setup - imports and database connection |
| 2 | Quick table overview with row counts |
| 3 | Data Completeness - newest/oldest 5 days + NULL flagging |
| 4 | Multi-Table Health Summary - freshness check |
| 5 | Consolidated Sanity Checks - duplicates, invalid values |
| 6 | Cleanup - close connection |

**Usage:** Change `check_symbol` in Cell 3 to test different tickers.

In [1]:
# Cell 1: Setup - imports and database connection
import sys
from pathlib import Path

# Add darkpool root to path for darkpool_analysis imports
darkpool_root = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
sys.path.insert(0, str(darkpool_root))

from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)

In [2]:
# Cell 2: Quick Table Overview - all tables with row counts
from IPython.display import display

overview = conn.execute("""
    SELECT name as table_name
    FROM (SHOW TABLES)
""").df()

# Add row counts
row_counts = []
for tbl in overview['table_name']:
    cnt = conn.execute(f"SELECT COUNT(*) as n FROM {tbl}").df()['n'].iloc[0]
    row_counts.append(cnt)
overview['rows'] = row_counts
display(overview)

,table_name,rows
0,composite_signal,0
1,daily_metrics,9828
2,finra_otc_weekly_raw,1948
3,finra_short_daily_all_raw,0
4,finra_short_daily_raw,9789
5,index_constituent_short_agg_daily,0
6,lit_direction_daily,10149
7,options_premium_daily,3170
8,options_premium_summary,0
9,polygon_daily_agg_raw,78


In [3]:
# Cell 3: Data Completeness Check - newest/oldest 5 days with NULL field flagging

check_symbol = "AAPL"  # Change to test different tickers

print(f"{'='*60}")
print(f"DATA COMPLETENESS CHECK FOR: {check_symbol}")
print(f"{'='*60}")

# Get newest 5 days
print("\n📅 NEWEST 5 DAYS (daily_metrics):")
newest = conn.execute("""
    SELECT * FROM daily_metrics
    WHERE symbol = ?
    ORDER BY date DESC
    LIMIT 5
""", [check_symbol]).df()
display(newest)

# Get oldest 5 days
print("\n📅 OLDEST 5 DAYS (daily_metrics):")
oldest = conn.execute("""
    SELECT * FROM daily_metrics
    WHERE symbol = ?
    ORDER BY date ASC
    LIMIT 5
""", [check_symbol]).df()
display(oldest)

# Combine for NULL analysis
combined = pd.concat([newest, oldest], ignore_index=True)

# Count NULLs per column
print("\n🔍 NULL FIELD ANALYSIS (across newest + oldest 10 rows):")
null_counts = combined.isnull().sum()
total_rows = len(combined)

# Create summary dataframe
null_summary = pd.DataFrame({
    'column': null_counts.index,
    'null_count': null_counts.values,
    'total_rows': total_rows,
    'null_pct': (null_counts.values / total_rows * 100).round(1),
    'status': ['⚠️ MISSING' if n > 0 else '✅ OK' for n in null_counts.values]
})

# Show only columns with issues first, then OK columns
null_summary_sorted = null_summary.sort_values(['null_count', 'column'], ascending=[False, True])
display(null_summary_sorted)

# Summary flag
missing_cols = null_summary[null_summary['null_count'] > 0]['column'].tolist()
if missing_cols:
    print(f"\n⚠️  ALERT: {len(missing_cols)} columns have NULL values:")
    for col in missing_cols:
        pct = null_summary[null_summary['column'] == col]['null_pct'].values[0]
        print(f"   - {col}: {pct}% null")
else:
    print("\n✅ ALL FIELDS POPULATED - No NULL values detected")

DATA COMPLETENESS CHECK FOR: AAPL

📅 NEWEST 5 DAYS (daily_metrics):


,date,symbol,log_buy_sell,short_volume,short_exempt_volume,short_total_volume,short_buy_volume,short_sell_volume,short_ratio,short_ratio_z,...,accumulation_score_display,confidence,data_quality,has_otc,has_short,has_lit,has_price,pressure_context_label,inference_version,accumulation_short_z_source
0,2025-12-31,AAPL,0.684242,2429403.0,17801.0,7595949.0,2429403.0,5166546.0,0.319829,-0.726710,...,46.821333,0.56,PRE_OTC,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
1,2025-12-30,AAPL,0.952649,2157509.0,41989.0,7435241.0,2157509.0,5277732.0,0.290173,-1.157639,...,45.720349,0.56,PRE_OTC,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
2,2025-12-29,AAPL,-0.084380,2221533.0,43866.0,6852213.0,2221533.0,4630680.0,0.324207,-0.878838,...,34.712775,0.56,PRE_OTC,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
3,2025-12-26,AAPL,-0.204466,2113861.0,19048.0,6896691.0,2113861.0,4782830.0,0.306504,-1.174198,...,31.859550,0.56,PRE_OTC,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
4,2025-12-24,AAPL,-0.643543,2766671.0,22134.0,7213936.0,2766671.0,4447265.0,0.383518,-0.413228,...,33.900752,0.56,PRE_OTC,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z



📅 OLDEST 5 DAYS (daily_metrics):


,date,symbol,log_buy_sell,short_volume,short_exempt_volume,short_total_volume,short_buy_volume,short_sell_volume,short_ratio,short_ratio_z,...,accumulation_score_display,confidence,data_quality,has_otc,has_short,has_lit,has_price,pressure_context_label,inference_version,accumulation_short_z_source
0,2024-12-31,AAPL,0.154592,5227904.0,46625.0,11250398.0,5227904.0,6022494.0,0.464686,NaN,...,NaN,1.0,OTC_ANCHORED,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
1,2025-01-02,AAPL,0.381936,11000653.0,67952.0,22651137.0,11000653.0,11650484.0,0.485656,NaN,...,NaN,1.0,OTC_ANCHORED,True,True,True,True,Neutral,PhaseA_v1,short_buy_sell_ratio_z
2,2025-01-03,AAPL,0.573388,5580044.0,45789.0,14712243.0,5580044.0,9132199.0,0.379279,NaN,...,NaN,0.8,OTC_ANCHORED,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
3,2025-01-06,AAPL,-0.308138,7488684.0,43148.0,14269140.0,7488684.0,6780456.0,0.524817,NaN,...,NaN,0.8,OTC_ANCHORED,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z
4,2025-01-07,AAPL,-0.477921,6567191.0,21047.0,13307765.0,6567191.0,6740574.0,0.493486,0.486678,...,47.259412,0.8,OTC_ANCHORED,True,True,True,False,Neutral,PhaseA_v1,short_buy_sell_ratio_z



🔍 NULL FIELD ANALYSIS (across newest + oldest 10 rows):


,column,null_count,total_rows,null_pct,status
45,otc_participation_delta,10,10,100.0,⚠️ MISSING
44,otc_participation_z,10,10,100.0,⚠️ MISSING
33,return_z,10,10,100.0,⚠️ MISSING
32,return_1d,9,10,90.0,⚠️ MISSING
27,close,8,10,80.0,⚠️ MISSING
29,high,8,10,80.0,⚠️ MISSING
30,low,8,10,80.0,⚠️ MISSING
34,range_pct,8,10,80.0,⚠️ MISSING
31,volume,8,10,80.0,⚠️ MISSING
28,vwap,8,10,80.0,⚠️ MISSING



⚠️  ALERT: 20 columns have NULL values:
   - short_ratio_z: 40.0% null
   - short_buy_sell_ratio_z: 40.0% null
   - vwbr_z: 40.0% null
   - lit_buy_ratio_z: 40.0% null
   - lit_flow_imbalance_z: 40.0% null
   - close: 80.0% null
   - vwap: 80.0% null
   - high: 80.0% null
   - low: 80.0% null
   - volume: 80.0% null
   - return_1d: 90.0% null
   - return_z: 100.0% null
   - range_pct: 80.0% null
   - otc_buy_ratio_z: 40.0% null
   - weekly_total_volume: 70.0% null
   - otc_participation_rate: 70.0% null
   - otc_participation_z: 100.0% null
   - otc_participation_delta: 100.0% null
   - accumulation_score: 40.0% null
   - accumulation_score_display: 40.0% null


In [4]:
# Cell 4: Multi-Table Data Health Summary - check all key tables for sample ticker
print(f"{'='*60}")
print(f"MULTI-TABLE HEALTH CHECK FOR: {check_symbol}")
print(f"{'='*60}")

# Define tables and their date columns
table_checks = [
    ("daily_metrics", "date", "symbol"),
    ("lit_direction_daily", "date", "symbol"),
    ("finra_short_daily_raw", "trade_date", "symbol"),
    ("polygon_daily_agg_raw", "trade_date", "symbol"),
]

health_rows = []

for table, date_col, sym_col in table_checks:
    try:
        result = conn.execute(f"""
            SELECT 
                COUNT(*) as row_count,
                MIN({date_col}) as oldest_date,
                MAX({date_col}) as newest_date
            FROM {table}
            WHERE {sym_col} = ?
        """, [check_symbol]).df()
        
        row_count = result['row_count'].iloc[0]
        oldest = result['oldest_date'].iloc[0]
        newest = result['newest_date'].iloc[0]
        
        # Check if newest date is within last 7 days
        if newest and pd.Timestamp(newest) >= pd.Timestamp.now() - pd.Timedelta(days=7):
            freshness = "✅ FRESH"
        elif newest:
            freshness = "⚠️ STALE"
        else:
            freshness = "❌ NO DATA"
            
        health_rows.append({
            'table': table,
            'rows': row_count,
            'oldest': oldest,
            'newest': newest,
            'freshness': freshness
        })
    except Exception as e:
        health_rows.append({
            'table': table,
            'rows': 0,
            'oldest': None,
            'newest': None,
            'freshness': f"❌ ERROR: {str(e)[:30]}"
        })

health_df = pd.DataFrame(health_rows)
display(health_df)

# Overall verdict
stale_tables = health_df[health_df['freshness'].str.contains('STALE|ERROR|NO DATA', na=False)]['table'].tolist()
if stale_tables:
    print(f"\n⚠️  ATTENTION: {len(stale_tables)} table(s) may need refresh:")
    for t in stale_tables:
        print(f"   - {t}")
else:
    print("\n✅ ALL TABLES HAVE FRESH DATA")

MULTI-TABLE HEALTH CHECK FOR: AAPL


,table,rows,oldest,newest,freshness
0,daily_metrics,252,2024-12-31,2025-12-31,✅ FRESH
1,lit_direction_daily,251,2024-12-31,2025-12-31,✅ FRESH
2,finra_short_daily_raw,251,2024-12-31,2025-12-31,✅ FRESH
3,polygon_daily_agg_raw,2,2024-12-31,2025-01-02,⚠️ STALE



⚠️  ATTENTION: 1 table(s) may need refresh:
   - polygon_daily_agg_raw


In [5]:
# Cell 5: Consolidated Sanity Checks - duplicates and invalid values
print(f"{'='*60}")
print("SANITY CHECKS")
print(f"{'='*60}")

# Check 1: Duplicate rows in daily_metrics
print("\n🔍 Duplicate rows in daily_metrics:")
dupes_dm = conn.execute("""
    SELECT symbol, date, COUNT(*) AS n
    FROM daily_metrics
    GROUP BY symbol, date
    HAVING n > 1
""").df()
if len(dupes_dm) == 0:
    print("   ✅ No duplicates found")
else:
    print(f"   ⚠️ {len(dupes_dm)} duplicate(s) found:")
    display(dupes_dm)

# Check 2: Duplicate rows in lit_direction_daily
print("\n🔍 Duplicate rows in lit_direction_daily:")
dupes_lit = conn.execute("""
    SELECT symbol, date, COUNT(*) AS n
    FROM lit_direction_daily
    GROUP BY symbol, date
    HAVING n > 1
""").df()
if len(dupes_lit) == 0:
    print("   ✅ No duplicates found")
else:
    print(f"   ⚠️ {len(dupes_lit)} duplicate(s) found:")
    display(dupes_lit)

# Check 3: Invalid short ratios (outside 0-1 range)
print("\n🔍 Invalid short ratios (outside 0-1):")
bad_ratios = conn.execute("""
    SELECT symbol, date, short_ratio
    FROM daily_metrics
    WHERE short_ratio IS NOT NULL AND (short_ratio < 0 OR short_ratio > 1)
    LIMIT 10
""").df()
if len(bad_ratios) == 0:
    print("   ✅ All short ratios valid")
else:
    print(f"   ⚠️ Invalid ratios found:")
    display(bad_ratios)

# Check 4: Zero/negative lit volumes with non-null ratios
print("\n🔍 Invalid lit volumes (zero/negative with computed ratio):")
bad_lit = conn.execute("""
    SELECT symbol, date, lit_buy_volume, lit_sell_volume
    FROM lit_direction_daily
    WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
      AND log_buy_sell IS NOT NULL
    LIMIT 10
""").df()
if len(bad_lit) == 0:
    print("   ✅ All lit volumes valid")
else:
    print(f"   ⚠️ Invalid volumes found:")
    display(bad_lit)

print("\n" + "="*60)

SANITY CHECKS

🔍 Duplicate rows in daily_metrics:
   ✅ No duplicates found

🔍 Duplicate rows in lit_direction_daily:
   ✅ No duplicates found

🔍 Invalid short ratios (outside 0-1):
   ✅ All short ratios valid

🔍 Invalid lit volumes (zero/negative with computed ratio):
   ✅ All lit volumes valid



In [6]:
# Cell 6: Cleanup - close database connection
conn.close()
print("Database connection closed.")

Database connection closed.
